In [ ]:
import csv
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [ ]:


# --- Function to extract data for 4th structure from a page ---
def extract_company_info(driver):
    data = {}
    
    def safe_find_text(by, selector):
        try:
            return driver.find_element(by, selector).text.strip()
        except NoSuchElementException:
            return ""
    
    # Extract all key-value blocks inside the container
    # Blocks with label and value classes
    
    blocks = driver.find_elements(By.CSS_SELECTOR, "div.block-QCJM7wcY")
    for block in blocks:
        try:
            label = block.find_element(By.CSS_SELECTOR, "div.label-QCJM7wcY").text.strip()
            # The value could be inside an <a> or <div> with class value-QCJM7wcY
            try:
                value = block.find_element(By.CSS_SELECTOR, "a .value-QCJM7wcY").text.strip()
            except NoSuchElementException:
                value = block.find_element(By.CSS_SELECTOR, "div.value-QCJM7wcY").text.strip()
            
            data[label] = value
        except NoSuchElementException:
            continue
    
    # Extract the about/description paragraph (last div with content-H16icEW0)
    try:
        description = driver.find_element(By.CSS_SELECTOR, "div.container-H16icEW0 div.content-H16icEW0 span").text.strip()
        data['Description'] = description
    except NoSuchElementException:
        data['Description'] = ""
    
    return data

# --- Main scraping logic ---
def main():
    # Chrome options to run headless (optional)
    options = Options()
    # options.add_argument("--headless")  # Uncomment if you want no browser UI
    
    driver = webdriver.Chrome(options=options)

    # Read links from CSV file (symbol_link.csv)
    links = []
    with open("symbol_link.csv", "r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        # Assuming the CSV has links in first column; skip header if any
        for row in reader:
            if row and "http" in row[0]:
                links.append(row[0])

    all_data = []
    
    for link in links:
        print(f"Processing: {link}")
        driver.get(link)
        time.sleep(3)  # Wait for page load; adjust as needed

        company_data = extract_company_info(driver)
        company_data['URL'] = link  # Save source link for reference
        all_data.append(company_data)

    driver.quit()

    # Save all extracted data to CSV
    df = pd.DataFrame(all_data)
    df.to_csv("extracted_company_info.csv", index=False, encoding="utf-8")
    print("Scraping done and data saved to extracted_company_info.csv")

if __name__ == "__main__":
    main()



In [ ]:
symbol_links_df = pd.read_csv("symbol_link.csv")
exchange_list = symbol_links_df["Exchange"].tolist()
link_list = symbol_links_df["Symbol"].tolist()

print(exchange_list)
print(link_list)
driver = webdriver.Chrome()


In [ ]:

def sanitize_text(text):
    # Remove special unicode characters and extra whitespace
    cleaned = re.sub(r'[^\x00-\x7F]+', '', text)  # keep only ASCII
    return cleaned.strip()
company_info = []
for i in range(len(link_list)):
    url = f"https://www.tradingview.com/symbols/{exchange_list[i]}-{link_list[i]}/"
    driver.get(url)
    print(url)
    time.sleep(3)

    records = driver.find_elements(By.CSS_SELECTOR, ".wrapper-QCJM7wcY")

    
    for record in records:
        children = record.find_elements(By.CLASS_NAME, "apply-overflow-tooltip")

        for j in range(min(23, len(children))):
            raw_text = children[j].text
            cleaned_text = sanitize_text(raw_text)
            company_info.append(cleaned_text)

  

print(company_info)